In [1]:
import os
from google.cloud import storage
from PIL import Image
from io import BytesIO
import pandas as pd

client = storage.Client(project='prime-basis-374911')
bucket = client.get_bucket('kitchen-ai-images')

In [ ]:
#define which food to pull from bucket
classes_to_include = [
"apple", "avocado", "banana", 
"grape", "kiwi", "lemon",
"lime", "orange", "blueberry",
"pineapple", "strawberry",

"capsicum", "broccoli",
"cabbage", "carrot",
"cauliflower", "cucumber",
"potato", "tomato",
"eggplant", "zucchini",
"mushroom", "chicken",
"beef", "pork",
"white", "milk",
"lettuce", "celery",
"onion"]

In [ ]:
def save_bucket_data_to(miniset=False):
    all_files = bucket.list_blobs()
    
    #save to here, in train/veg/veg_1.jpg
    path_to_data_folder = os.path.join("..", "..", "test_data", "17_250_no_lime")
    
    class_limit = 501 #not really used, now split among below vars
    class_count = 0
    train_count = 0
    test_count = 0
    val_count = 0
    
    train_max = 250
    test_max = 25
    val_max = 25
    
    current_class = ""
    
    for this_file in all_files:
        und_split = this_file.name.split('_')
        dot_split = this_file.name.split('.')
        food_bucket_path = this_file.path #to track old names

        #get info from bucket path name
        food_class = und_split[0].lower()
        
        if food_class == "white":
            food_class = "white_wine" 
        
        file_ext = dot_split[1] #print(img_ext)
        file_num = dot_split[0].split('_')[-1] #print(file_num)
        
        #data_split = "train" "test" "val"
        data_split = dot_split[0].split('_')[-2]      
        
        #make sure the folder is called val
        if data_split == "validation":
            data_split = "val"
        
        #create path to correct folder
        path_to_dataset = os.path.join(path_to_data_folder, data_split)

        #tracking, if new class encountered then output prev. totals.
        if food_class != current_class:
            #if statement avoids printing empty for first
            if current_class:
                print(f"loaded total {class_count} of {current_class}")
                print(f"{train_count} train")
                print(f"{val_count} val")
                print(f"{test_count} test")
            
            print(f"loading {food_class}...")
            current_class = food_class
            class_count = 0
            train_count, test_count, val_count = 0,0,0
                        
        #create new file path
        new_filename = food_class + "_" + str(class_count) + ".jpg" #"." + file_ext
        new_dir_path = os.path.join(path_to_dataset, food_class) #used to create folder if not already existing
        new_path = os.path.join(new_dir_path,new_filename)   #class/class-n.ext
        
        ## STOPPING / SKIPPING CONDITIONS
        
        if food_class not in classes_to_include:
            continue
 
        
        if data_split == "train" and (train_count >= train_max):
            continue
        if data_split == "test"  and (test_count >= test_max):
            continue
        if data_split == "val"   and (val_count >= val_max):
            continue            
            
        try:
            #get image from bucket
            file_image = Image.open(BytesIO(this_file.download_as_bytes()))
            #convert to rgb ==> jpg
            rgb_image = file_image.convert('RGB')

            #save image
            os.makedirs(new_dir_path, exist_ok=True) #True means don't create new dir if already exists #os.makedirs(path, exist_ok=True)
            test_img = rgb_image.save(new_path)

            class_count += 1
            
            if data_split == "train":
                train_count += 1
            if data_split == "test":
                test_count += 1
            if data_split == "val":
                val_count += 1
            
        except:
            print(f"error loading from {food_bucket_path} \n to {new_path}")
    
    print(f"loaded total {class_count} of {current_class}")
    print(f"{train_count} train")
    print(f"{val_count} val")
    print(f"{test_count} test")
    print("done!")


In [ ]:
save_bucket_data_to()